# Propositionalization: Interstate 94

In this notebbok, we compare getML's FastProp against well-known feature engineering libraries featuretools and tsfresh.

Summary:

- Prediction type: __Regression model__
- Domain: __Transportation__
- Prediction target: __Hourly traffic volume__ 
- Source data: __Multivariate time series, 5 components__
- Population size: __24096__

# Background

A common approach to feature engineering is to generate attribute-value representations from relational data by applying a fixed set of aggregations to columns of interest and perform a feature selection on the (possibly large) set of generated features afterwards. In academia, this approach is called _propositionalization._

getML's [FastProp](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html#fastprop) is an implementation of this propositionalization approach that has been optimized for speed and memory efficiency. In this notebook, we want to demonstrate how – well – fast FastProp is. To this end, we will benchmark FastProp against the popular feature engineering libraries [featuretools](https://www.featuretools.com/) and [tsfresh](https://tsfresh.readthedocs.io/en/latest/). Both of these libraries use propositionalization approaches for feature engineering.

In this notebook, we predict the hourly traffic volume on I-94 westbound from Minneapolis-St Paul. The analysis is built on top of a dataset provided by the [MN Department of Transportation](https://www.dot.state.mn.us), with some data preparation done by [John Hogue](https://github.com/dreyco676/Anomaly_Detection_A_to_Z/). For further details about the data set refer to [the full notebook](../interstate94.ipynb).

# Analysis

## Table of contents

1. [Loading data](#1.-Loading-data)
2. [Predictive modeling](#2.-Predictive-modeling)
3. [Comparison](#3.-Comparison)

Let's get started with the analysis and set-up your session:

In [50]:
import datetime
import os
import sys
import time

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from IPython.display import Image

plt.style.use("seaborn")
%matplotlib inline

import getml

print(f"getML API version: {getml.__version__}\n")

getml.engine.set_project("interstate94")

getML API version: 1.2.0



Loading pipelines...
[========================================] 100%


Connected to project 'interstate94'


In [51]:
sys.path.append(os.path.join(sys.path[0], ".."))

from utils import Benchmark, FTTimeSeriesBuilder, TSFreshBuilder

## 1. Loading data

### 1.1 Download from source

We begin by downloading the data from the UC Irvine Machine Learning repository:

In [52]:
traffic = getml.datasets.load_interstate94(roles=True, units=True)


Loading traffic...
[========================================] 100%


In [53]:
traffic.set_role(traffic.roles.categorical, getml.data.roles.unused_string)

In [54]:
traffic

name,ds,traffic_volume,holiday,day,month,weekday,hour,year
role,time_stamp,target,unused_string,unused_string,unused_string,unused_string,unused_string,unused_string
unit,time stamp,,,day,month,weekday,hour,year
0,2016-01-01,1513,New Years Day,1,1,4,0,2016
1,2016-01-01 01:00:00,1550,New Years Day,1,1,4,1,2016
2,2016-01-01 02:00:00,993,New Years Day,1,1,4,2,2016
3,2016-01-01 03:00:00,719,New Years Day,1,1,4,3,2016
4,2016-01-01 04:00:00,533,New Years Day,1,1,4,4,2016
,...,...,...,...,...,...,...,...
24091,2018-09-30 19:00:00,3543,No holiday,30,9,6,19,2018
24092,2018-09-30 20:00:00,2781,No holiday,30,9,6,20,2018


### 1.2 Define relational model


In [80]:
split = getml.data.split.time(traffic, "ds", test=getml.data.time.datetime(2018, 3, 15))

In [87]:
time_series = getml.data.TimeSeries(
    population=traffic,
    split=split,
    alias="traffic",
    time_stamps="ds",
    horizon=getml.data.time.hours(1),
    memory=getml.data.time.hours(24),
    lagged_targets=True,
)

time_series

data model

  traffic:
    columns:
    - traffic_volume: target
    - ds: time_stamp
    - holiday: unused_string
    - day: unused_string
    - month: unused_string
    - ...

    joins:
    - right: 'traffic'
      time_stamps: (traffic.ds, traffic.ds)
      relationship: 'many-to-many'
      memory: 86400.0
      horizon: 3600.0
      lagged_targets: True

  traffic:
    columns:
    - traffic_volume: target
    - ds: time_stamp
    - holiday: unused_string
    - day: unused_string
    - month: unused_string
    - ...


container

  population
      subset   name       rows   type
  0   test     traffic    4800   View
  1   train    traffic   19296   View

  peripheral
      name       rows   type     
  0   traffic   24096   DataFrame

## 2. Predictive modeling

We loaded the data, defined the roles, units and the abstract data model. Next, we create a getML pipeline for relational learning.

### 2.1 Propositionalization with getML's FastProp

In [8]:
seasonal = getml.preprocessors.Seasonal()

fast_prop = getml.feature_learning.FastProp(
    loss_function=getml.feature_learning.loss_functions.SquareLoss,
    num_threads=1,
)

__Build the pipeline__

In [9]:
pipe_fp_fl = getml.pipeline.Pipeline(
    preprocessors=[seasonal],
    feature_learners=[fast_prop],
    data_model=time_series.data_model,
    tags=["feature learning", "fastprop"],
)

pipe_fp_fl

Pipeline(data_model='traffic',
         feature_learners=['FastProp'],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=['traffic'],
         predictors=[],
         preprocessors=['Seasonal'],
         share_selected_features=0.5,
         tags=['feature learning', 'fastprop'])

In [10]:
pipe_fp_fl.check(time_series.train)

Checking data model...


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

Checking...
[========================================] 100%


OK.


In [11]:
benchmark = Benchmark()

In [12]:
with benchmark("fastprop"):
    pipe_fp_fl.fit(time_series.train)
    fastprop_train = pipe_fp_fl.transform(time_series.train, df_name="fastprop_train")

Checking data model...


Staging...
[========================================] 100%


OK.


Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Trying 461 features...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:18.279971



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




In [13]:
fastprop_test = pipe_fp_fl.transform(time_series.test, df_name="fastprop_test")



Staging...
[========================================] 100%

Preprocessing...
[========================================] 100%

FastProp: Building features...
[========================================] 100%




In [14]:
predictor = getml.predictors.XGBoostRegressor()

pipe_fp_pr = getml.pipeline.Pipeline(
    tags=["prediction", "fastprop"], predictors=[predictor]
)

In [15]:
pipe_fp_pr.fit(fastprop_train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:10.281889



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'fastprop'])

url: http://localhost:1709/#/getpipeline/interstate94/Mxcuea/0/

In [16]:
pipe_fp_pr.score(fastprop_test)



Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-11-10 12:11:26,fastprop_train,traffic_volume,200.374,292.5448,0.9779
1,2021-11-10 12:11:26,fastprop_test,traffic_volume,185.2662,266.3327,0.9823


### 2.2 Propositionalization with featuretools

In [17]:
traffic_train = time_series.train.population
traffic_test = time_series.test.population

In [18]:
dfs_pandas = {}

for df in [traffic_train, traffic_test, traffic]:
    dfs_pandas[df.name] = df.drop(df.roles.unused).to_pandas()
    dfs_pandas[df.name]["join_key"] = 1

In [19]:
ft_builder = FTTimeSeriesBuilder(
    num_features=200,
    horizon=pd.Timedelta(hours=1),
    memory=pd.Timedelta(hours=24),
    column_id="join_key",
    time_stamp="ds",
    target="traffic_volume",
    allow_lagged_targets=True,
)

In [20]:
with benchmark("featuretools"):
    featuretools_train = ft_builder.fit(dfs_pandas["train"])

featuretools_test = ft_builder.transform(dfs_pandas["test"])

featuretools: Trying features...


/home/patrick/anaconda3/lib/python3.8/site-packages/featuretools/synthesis/dfs.py:309: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'count', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


Selecting the best out of 59 features...
Time taken: 0h:2m:59.172488



/home/patrick/anaconda3/lib/python3.8/site-packages/featuretools/synthesis/dfs.py:309: UnusedPrimitiveWarning: Some specified primitives were not used during DFS:
  agg_primitives: ['all', 'any', 'count', 'num_true', 'percent_true']
This may be caused by a using a value of max_depth that is too small, not setting interesting values, or it may indicate no compatible columns for the primitive were found in the data.
  warnings.warn(warning_msg, UnusedPrimitiveWarning)


In [21]:
roles = {
    getml.data.roles.join_key: ["join_key"],
    getml.data.roles.target: ["traffic_volume"],
    getml.data.roles.time_stamp: ["ds"],
}

df_featuretools_train = getml.data.DataFrame.from_pandas(
    featuretools_train, name="featuretools_train", roles=roles
)

df_featuretools_test = getml.data.DataFrame.from_pandas(
    featuretools_test, name="featuretools_test", roles=roles
)

In [22]:
df_featuretools_train.set_role(
    df_featuretools_train.roles.unused, getml.data.roles.numerical
)

df_featuretools_test.set_role(
    df_featuretools_test.roles.unused, getml.data.roles.numerical
)

In [23]:
predictor = getml.predictors.XGBoostRegressor()

pipe_ft_pr = getml.pipeline.Pipeline(
    tags=["prediction", "featuretools"], predictors=[predictor]
)

pipe_ft_pr

Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

In [24]:
pipe_ft_pr.check(df_featuretools_train)

Checking data model...


Staging...
[========================================] 100%

Checking...
[========================================] 100%


OK.


In [25]:
pipe_ft_pr.fit(df_featuretools_train)

Checking data model...


Staging...
[========================================] 100%


OK.


Staging...
[========================================] 100%

XGBoost: Training as predictor...
[========================================] 100%


Trained pipeline.
Time taken: 0h:0m:1.997418



Pipeline(data_model='population',
         feature_learners=[],
         feature_selectors=[],
         include_categorical=False,
         loss_function=None,
         peripheral=[],
         predictors=['XGBoostRegressor'],
         preprocessors=[],
         share_selected_features=0.5,
         tags=['prediction', 'featuretools'])

url: http://localhost:1709/#/getpipeline/interstate94/wUrR1j/0/

In [26]:
pipe_ft_pr.score(df_featuretools_test)



Staging...
[========================================] 100%




,date time,set used,target,mae,rmse,rsquared
0,2021-11-10 12:15:13,featuretools_train,traffic_volume,217.4832,317.1563,0.974
1,2021-11-10 12:15:13,featuretools_test,traffic_volume,209.5696,330.5634,0.9724


### 2.3 Propositionalization with tsfresh

tsfresh failed to run through due to an apparent bug in the tsfresh library and is therefore excluded from this analysis.

## 3. Comparison

In [30]:
num_features = dict(
    fastprop=461,
    featuretools=59,
)

runtime_per_feature = [
    benchmark.runtimes["fastprop"] / num_features["fastprop"],
    benchmark.runtimes["featuretools"] / num_features["featuretools"],
]

features_per_second = [1.0 / r.total_seconds() for r in runtime_per_feature]

normalized_runtime_per_feature = [
    r / runtime_per_feature[0] for r in runtime_per_feature
]

comparison = pd.DataFrame(
    dict(
        runtime=[benchmark.runtimes["fastprop"], benchmark.runtimes["featuretools"]],
        num_features=num_features.values(),
        features_per_second=features_per_second,
        normalized_runtime=[
            1,
            benchmark.runtimes["featuretools"] / benchmark.runtimes["fastprop"],
        ],
        normalized_runtime_per_feature=normalized_runtime_per_feature,
        rsquared=[pipe_fp_pr.rsquared, pipe_ft_pr.rsquared],
        rmse=[pipe_fp_pr.rmse, pipe_ft_pr.rmse],
        mae=[pipe_fp_pr.mae, pipe_ft_pr.mae],
    )
)

comparison.index = ["getML: FastProp", "featuretools"]

In [31]:
comparison

,runtime,num_features,features_per_second,normalized_runtime,normalized_runtime_per_feature,rsquared,rmse,mae
getML: FastProp,0 days 00:00:25.596076,461,18.010554,1.000000,1.000000,0.982251,266.332662,185.266245
featuretools,0 days 00:02:59.173175,59,0.329290,7.000025,54.695045,0.972389,330.563417,209.569640


In [32]:
comparison.to_csv("comparisons/interstate94.csv")

## Why is FastProp so fast?

First, FastProp hugely benefits from getML's custom-built C++-native in-memory database engine. The engine is highly optimized for working with relational data structures and makes use of information about the relational structure of the data to efficiently store and carry out computations on such data. This matters in particular for time series where we [relate the current observation to a certain number of observations from the past](https://docs.getml.com/latest/user_guide/data_model/data_model.html#time-series): Other libraries have to deal explicitly with this inherent structure of (multivariate) time series; and such explicit transformations are costly, in terms of consumption of both, memory and computational resources. All operations on data stored in getML's engine benefit from implementations in modern C++. Further, we are taking advantage of functional design patterns where all column-based operations are evaluated lazily. So, for example, aggregations are carried out only on rows that matter (taking into account even complex conditions that might span multiple tables in the relational model). Duplicate operations are reduced to a bare minimum by keeping track of the relational data model. In addition to the mere advantage in performance, FastProp, by building on an abstract data model, also has an edge in memory consumption based on the abstract database design, the reliance on efficient storage patterns (utilizing pointers and indices) for concrete data, and by taking advantage of functional design patterns and lazy computations. This allows working with data sets of substantial size even without falling back to distributed computing models.

# Next Steps

If you are interested in further real-world applications of getML, visit the [notebook section on getml.com](https://notebooks.getml.com/). If you want to gain a deeper understanding about our notebooks' contents or download the code behind the notebooks, have a look at the [getml-demo repository](https://github.com/getml/getml-demo/). Here, you can also find [futher benchmarks of getML](https://github.com/getml/getml-demo/#benchmarks).

Want to try out without much hassle? Just head to [try.getml.com](https://try.getml.com) to launch an instance of getML directly in your browser.

Further, here is some additional material from our [documentation](https://docs.getml.com/latest/) if you want to learn more about getML:
* [Annotating data within getML's data frames](https://docs.getml.com/latest/user_guide/annotating_data/annotating_data.html),
* [Defining your relational structure through getML's abstract data model](https://docs.getml.com/latest/user_guide/data_model/data_model.html), or
* [An introduction to feature learning](https://docs.getml.com/latest/user_guide/feature_engineering/feature_engineering.html).

# Get in contact

If you have any questions, just write us an [email](https://getml.com/contact/lets-talk/). Prefer a private demo of getML for your team? Just [contact us](https://getml.com/contact/lets-talk/) to arrange an introduction to getML.